In [1]:
# ||||||| RUN THIS ONE SETUP CELL IN EVERY NEW NOTEBOOK |||||||

from openai import OpenAI

# Initialize the client to connect to OpenRouter
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",   # <-- REPLACE THIS WITH YOUR REAL KEY!
)

# Headers that OpenRouter requires
extra_headers = {
    "HTTP-Referer": "http://localhost:8888",
    "X-Title": "Jupyter AI Assistant",
}

print("✅ Setup complete! Ready to use DeepSeek for free via OpenRouter.")

✅ Setup complete! Ready to use DeepSeek for free via OpenRouter.


In [2]:
# === SUPPRESS OPENPYXL WARNINGS ===
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

# === STEP 1: LOAD THE NEW OUTBOUND DATA ===
# === INITIAL SETUP: RUN THIS CELL FIRST ===
import pandas as pd
import numpy as np

# Define the path to your new Inbound file
provreport_file_path = r"C:\Users\erictr\Documents\Python Billing Project\Provision_Report_Template.xlsx" # <<< CHANGE THIS TO YOUR ACTUAL FILE PATH

# Read the new file into a DataFrame called 'inbound_df'
provreport_df = pd.read_excel(provreport_file_path)

# Let's see what we're working with
print("✅ Provision Report Template Loaded Successfully!")
print(f"Shape: {provreport_df.shape} (rows, columns)")

✅ Provision Report Template Loaded Successfully!
Shape: (25, 8) (rows, columns)


In [3]:
# === STEP 2: UPDATE HELPER FUNCTION FOR INBOUND DATA ===

def ask_about_provreport(question):
    """
    A helper function to ask DeepSeek questions about the NEW Provision Report DataFrame.
    """
    
    data_context = f"""
    I am working with a NEW pandas DataFrame called `provreport_df`.

    Here is what the data looks like:
    - Shape: {provreport_df.shape} (rows, columns)
    - First 5 rows:
    {provreport_df.head().to_string()}
    
    - Column names and data types:
    {provreport_df.dtypes.to_string()}
    
    My question is: {question}
    """
    
    response = client.chat.completions.create(
        model="deepseek/deepseek-chat",
        messages=[
            {"role": "system", "content": "You are a expert data analyst and pandas specialist. Provide accurate, executable Python code."},
            {"role": "user", "content": data_context}
        ],
        extra_headers=extra_headers
    )
    
    return response.choices[0].message.content

print("✅ Helper function updated for 'provreport_df'!")

✅ Helper function updated for 'provreport_df'!


In [4]:
# Question 1: Basic Analysis
#answer = ask_deepseek("Are there any missing values in the 'TotalBilling' column? Show me the code to check. No explanation please.")
#print(answer)

In [5]:
# === STEP 2: LOAD MODIFIED OUTBOUND AND INBOUND DATA ===

# Define paths to your modified files
modified_outbound_path = r"C:\Users\erictr\Documents\Python Billing Project\Outbound Data\Result\Modified_Outbound_Data.xlsx"
modified_inbound_path = r"C:\Users\erictr\Documents\Python Billing Project\Inbound Data\Result\Modified_Inbound_Data.xlsx"

# Load the modified files
modified_outbound_df = pd.read_excel(modified_outbound_path)
modified_inbound_df = pd.read_excel(modified_inbound_path)

print("✅ Modified data files loaded successfully!")
print(f"Modified Outbound shape: {modified_outbound_df.shape}")
print(f"Modified Inbound shape: {modified_inbound_df.shape}")

✅ Modified data files loaded successfully!
Modified Outbound shape: (21420, 26)
Modified Inbound shape: (1830, 41)


In [6]:
import pandas as pd
from openpyxl import load_workbook
import shutil

# === Paths ===
provreport_file_path = r"C:\Users\erictr\Documents\Python Billing Project\Provision_Report_Template.xlsx"  # fixed template
final_output_path = r"C:\Users\erictr\Documents\Python Billing Project\Final_Provision_Report.xlsx"
opening_storage_path = r"C:\Users\erictr\Documents\Python Billing Project\Opening Storage Data\Monthly Stock Summary - 20250731.xlsx"

# === Copy template to final output ===
shutil.copy2(provreport_file_path, final_output_path)

# === Load previously modified data ===
modified_outbound_path = r"C:\Users\erictr\Documents\Python Billing Project\Outbound Data\Result\Modified_Outbound_Data.xlsx"
modified_inbound_path = r"C:\Users\erictr\Documents\Python Billing Project\Inbound Data\Result\Modified_Inbound_Data.xlsx"

modified_outbound_df = pd.read_excel(modified_outbound_path)
modified_inbound_df = pd.read_excel(modified_inbound_path)
opening_storage_df = pd.read_excel(opening_storage_path)

# === Append data to template ===
book = load_workbook(final_output_path)

with pd.ExcelWriter(final_output_path, engine='openpyxl', mode='a') as writer:
    writer._book = book
    writer._sheets = {ws.title: ws for ws in book.worksheets}
    modified_outbound_df.to_excel(writer, sheet_name="Outbound Data", index=False)
    modified_inbound_df.to_excel(writer, sheet_name="Inbound Data", index=False)
    opening_storage_df.to_excel(writer, sheet_name="Opening Storage Data", index=False)

print(f"✅ Final Provision Report saved to: {final_output_path}")
print("Sheets in file:", load_workbook(final_output_path).sheetnames)



✅ Final Provision Report saved to: C:\Users\erictr\Documents\Python Billing Project\Final_Provision_Report.xlsx
Sheets in file: ['Billing Summary', 'Inbound Details', 'Outbound Data', 'Inbound Data', 'Opening Storage Data']


Sheets in file: ['Billing Summary', 'Inbound Details', 'Outbound Data', 'Inbound Data', 'Opening Storage Data']


In [7]:
import pandas as pd
from openpyxl import load_workbook

# Full path to your Excel file
file_path = r"C:\Users\erictr\Documents\Python Billing Project\Final_Provision_Report.xlsx"

# Read the 'Opening Storage Data' tab
opening_storage = pd.read_excel(file_path, sheet_name='Opening Storage Data')

# Get values from Opening Storage Data
frozen_qty_value = opening_storage.iloc[5, 2] - opening_storage.iloc[1, 2]  # C6 - C3
chiller_qty_value = opening_storage.iloc[1, 2]  # C3

# Load workbook and select Summary sheet
wb = load_workbook(file_path)
ws = wb['Billing Summary']

# Write values to Summary sheet
ws['D2'] = frozen_qty_value   # Frozen
ws['D3'] = chiller_qty_value  # Chiller

# Save workbook
wb.save(file_path)

# Print the values
print("Frozen quantity value:", frozen_qty_value)
print("Chiller quantity value:", chiller_qty_value)

Frozen quantity value: 2980
Chiller quantity value: 10


In [8]:
import pandas as pd
from openpyxl import load_workbook

# Path to your Excel file
file_path = r"C:\Users\erictr\Documents\Python Billing Project\Final_Provision_Report.xlsx"

# Read the 'Outbound Data' tab
outbound_data = pd.read_excel(file_path, sheet_name='Outbound Data')

# Calculate sums
full_picking_qty_value = outbound_data['Full_Picking Count'].sum()
partial_qty_value = outbound_data['Partial_Picking Count'].sum()

# Load workbook and select Summary sheet
wb = load_workbook(file_path)
ws = wb['Billing Summary']

# Write values to Summary sheet
ws['D12'] = full_picking_qty_value   # Full_Picking
ws['D13'] = partial_qty_value  # Partial_Picking

# Save workbook
wb.save(file_path)

# Print the values
print("Full Picking quantity value:", full_picking_qty_value)
print("Partial Picking quantity value:", partial_qty_value)


Full Picking quantity value: 358
Partial Picking quantity value: 59527


In [9]:
import pandas as pd
from openpyxl import load_workbook

# Path to your Excel file
file_path = r"C:\Users\erictr\Documents\Python Billing Project\Final_Provision_Report.xlsx"

# Read 'Inbound Data' sheet
inbound_data = pd.read_excel(file_path, sheet_name='Inbound Data')

# Ensure the boolean column is actually boolean type
inbound_data['Inbound Qty > 8'] = inbound_data['Inbound Qty > 8'].astype(bool)

# 1️⃣ Count unique 'Date_PalletID_Location' where 'Inbound Qty > 8' is True
pallet_handling_in = inbound_data.loc[inbound_data['Inbound Qty > 8'] == True, 'Date_PalletID_Location'].nunique()

# 2️⃣ Sum 'Qty Received' only for rows where 'Inbound Qty > 8' is False
carton_handling_in = inbound_data.loc[inbound_data['Inbound Qty > 8'] == False, 'Inbound Qty (UOM)'].sum()

# Print values to check
print("Pallet Handling In (Full Inbound):", pallet_handling_in)
print("Carton Handling In (Partial Inbound):", carton_handling_in)

# Load workbook and select Summary sheet
wb = load_workbook(file_path)
ws = wb['Billing Summary']

# Write values to Summary sheet
ws['D10'] = pallet_handling_in      # Full_Inbound
ws['D11'] = carton_handling_in     # Partial_Inbound

# Save workbook
wb.save(file_path)


Pallet Handling In (Full Inbound): 1760
Carton Handling In (Partial Inbound): 63


In [10]:
import pandas as pd
from openpyxl import load_workbook

# Load Inbound Data
inbound_data = pd.read_excel(file_path, sheet_name='Inbound Data')

# Load workbook and select Summary sheet
wb = load_workbook(file_path)
ws = wb['Billing Summary']

# -------------------------
# 1H pallets
# -------------------------
FROZEN_pallet_1H = inbound_data.loc[
    (inbound_data['Storage Type'] == 'FROZEN') &
    (inbound_data['1H / 2H(Storage)'] == '1H'),
    'Date_PalletID_Location'
].nunique()

CHILLED_pallet_1H = inbound_data.loc[
    (inbound_data['Storage Type'] == 'CHILLED') &
    (inbound_data['1H / 2H(Storage)'] == '1H'),
    'Date_PalletID_Location'
].nunique()

# Write 1H to Summary
ws['D4'] = FROZEN_pallet_1H
ws['D6'] = CHILLED_pallet_1H

# -------------------------
# 2H pallets
# -------------------------
FROZEN_pallet_2H = inbound_data.loc[
    (inbound_data['Storage Type'] == 'FROZEN') &
    (inbound_data['1H / 2H(Storage)'] == '2H'),
    'Date_PalletID_Location'
].nunique()

CHILLED_pallet_2H = inbound_data.loc[
    (inbound_data['Storage Type'] == 'CHILLED') &
    (inbound_data['1H / 2H(Storage)'] == '2H'),
    'Date_PalletID_Location'
].nunique()

# Write 2H to Summary
ws['D5'] = FROZEN_pallet_2H
ws['D7'] = CHILLED_pallet_2H

# Save workbook
wb.save(file_path)

# Print to verify
print("1H Frozen:", FROZEN_pallet_1H)
print("1H Chilled:", CHILLED_pallet_1H)
print("2H Frozen:", FROZEN_pallet_2H)
print("2H Chilled:", CHILLED_pallet_2H)



1H Frozen: 879
1H Chilled: 9
2H Frozen: 884
2H Chilled: 7


In [11]:
import pandas as pd

# File path (Provision Report)
file_path = r"C:\Users\erictr\Documents\Python Billing Project\Final_Provision_Report.xlsx"

# Read Inbound Data (already has Sorting column from Modified_Inbound)
df_inbound = pd.read_excel(file_path, sheet_name="Inbound Data", engine="openpyxl")

# ✅ Keep only Import rows
df_inbound = df_inbound[df_inbound["01_Inbound Type"] == "Import"]

# ✅ Collapse by PO No (so only one row per PO No is kept)
df_po = df_inbound.groupby("PO No").first().reset_index()

# --- Build Inbound Details (per PO) ---
df_details = pd.DataFrame({
    "S/N": range(1, len(df_po) + 1),
    "01_Inbound Type": df_po["01_Inbound Type"],
    "PO No": df_po["PO No"],
    "Inbound Ref": df_po["Inbound Ref"],
    "In Date": df_po["Receiving"],
    "1H / 2H(Storage)": df_po["1H / 2H(Storage)"],
    "20FT-Palletised(Count)": 0,
    "20FT-Loose(Count)": df_po["Unstuffing Type"].apply(lambda x: 1 if x == "20 RF Container" else 0),
    "40FT-Palletised(Count)": 0,
    "40FT-Loose(Count)": df_po["Unstuffing Type"].apply(lambda x: 1 if x == "40 RF Container" else 0),
    "SFA Booking(Count)": df_po["SFA Booking"].apply(lambda x: 1 if x == "SFA" else 0),
    "SFA Physical Coordinaion (Count)": df_po["SFA Status"].apply(lambda x: 1 if x == "Passed" else 0),
    "Sorting Fee (6-10 marking) (Count)": df_po["Sorting"].apply(lambda x: 1 if x == "Sort" else 0)
})

# ✅ Sort by In Date (ascending)
df_details = df_details.sort_values(by="In Date", ascending=True).reset_index(drop=True)

# ✅ Re-generate S/N safely and keep as first column
df_details["S/N"] = range(1, len(df_details) + 1)
cols = ["S/N"] + [c for c in df_details.columns if c != "S/N"]
df_details = df_details[cols]

# Write back to Inbound Details tab
with pd.ExcelWriter(file_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df_details.to_excel(writer, sheet_name="Inbound Details", index=False)

print("✅ Inbound Details updated successfully (unique per PO No, sorted by In Date)!")


✅ Inbound Details updated successfully (unique per PO No, sorted by In Date)!


In [12]:
from openpyxl import load_workbook

# Load the workbook
wb = load_workbook(file_path)

# Read the Inbound Details sheet to get the sums
df_inbound_details = pd.read_excel(file_path, sheet_name="Inbound Details", engine="openpyxl")

# Calculate the sums
sum_20ft = df_inbound_details["20FT-Loose(Count)"].sum()
sum_40ft = df_inbound_details["40FT-Loose(Count)"].sum()
sum_sfa = df_inbound_details["SFA Booking(Count)"].sum()
sum_sfa_coor = df_inbound_details["SFA Physical Coordinaion (Count)"].sum()

# Write to Billing Summary sheet
ws_billing = wb["Billing Summary"]  # Make sure this sheet exists!

# Write sums to specific cells
ws_billing["D15"] = sum_20ft  # Sum of 20FT-Loose(Count)
ws_billing["D17"] = sum_40ft  # Sum of 40FT-Loose(Count)  
ws_billing["D19"] = sum_sfa   # Sum of SFA Booking(Count)
ws_billing["D20"] = sum_sfa_coor   # Sum of SFA Physical Coordinaion(Count)

# Save the workbook
wb.save(file_path)

print("✅ Billing Summary updated successfully!")
print(f"20FT-Loose Total: {sum_20ft} (written to D15)")
print(f"40FT-Loose Total: {sum_40ft} (written to D17)")
print(f"SFA Booking Total: {sum_sfa} (written to D19)")
print(f"SFA Booking Total: {sum_sfa_coor} (written to D20)")

✅ Billing Summary updated successfully!
20FT-Loose Total: 0 (written to D15)
40FT-Loose Total: 25 (written to D17)
SFA Booking Total: 10 (written to D19)
SFA Booking Total: 1 (written to D20)


In [13]:
from openpyxl import load_workbook

# ✅ Count unique PO No where Sorting = "Sort"
sort_pos = df_inbound.loc[df_inbound["Sorting"] == "Sort", "PO No"].nunique()

# ✅ Open workbook and write result to Billing Summary D18
wb = load_workbook(file_path)
ws = wb["Billing Summary"]
ws["D18"] = sort_pos

wb.save(file_path)